# Introduction to the Data Scraping Notebook

In this data scraping notebook, our objective is to gather relevant information from two key sources – The White House and The European Commission. By systematically collecting and processing data, we aim to provide valuable insights into the nature of their support, potential differences in rhetoric, and the impact of President Zelenskiy's visits.

The notebook includes pipelines for scraping the data from The White House and The European Commission. The scraping classes are implemented in a scr/scraper.py file.

# Set Up Environment

## Import necessary libraries

In [ ]:
import pandas as pd
import time

import src.scraper as s

# Scrape The White House Data

In this section, we retrieve data from The White House, focusing on President Biden's administration. This includes parsing official statements, speeches, and and press briefings. 

The data from The White House will be crucial in understanding the United States' stance and support for Ukraine, particularly in the context of President Zelenskiy's visits. It forms a foundational component of our comparative analysis with The European Commission.

## Scraping Pipeline

In [ ]:
# Initialize an empty dataframe to store the results
wh_articles_df = pd.DataFrame(columns=['Title', 'Date', 'Category', 'Location', 'Text'])

# List of categories to be scraped
wh_links = ['https://www.whitehouse.gov/briefing-room/speeches-remarks/',
            'https://www.whitehouse.gov/briefing-room/statements-releases/',
            'https://www.whitehouse.gov/briefing-room/press-briefings/']        

# Iterate through all categories
for link in wh_links:
    # Get the category
    category = link.split('/')[-2]
    print(category)
    # Initialize the scraping class
    scraper = s.TheWhiteHouseScraper(url=link)
    soup = scraper.get_html_content()
    
    # Get the total number of pages
    page_num = scraper.get_page_num(soup)
    print(f'Total number of pages: {page_num}')

    # Get articles from each page
    for i in range(1, page_num+1):
        page_link = f'{link}page/{i}/'
        page_scraper = s.TheWhiteHouseScraper(url=page_link)
        page_soup = page_scraper.get_html_content()
        
        # Add articles to a dataframe
        df_temp = pd.DataFrame(page_scraper.get_articles(page_soup, category))
        wh_articles_df = pd.concat([wh_articles_df, df_temp], ignore_index=True)

        # Print progress every 10%
        if i % (page_num // 10) == 0:
            print(f'{i}/{page_num} completed.')
    print()

print("Scraping completed.")

In [ ]:
wh_articles_df.shape

In [ ]:
wh_articles_df.head()

In [ ]:
# Save to csv
wh_articles_df.to_csv('data/thewhitehouse.csv', index=False)

# Scrape The European Commission Data

In this phase, our focus shifts to collecting data from The European Commission, which plays a significant role in the European Union's policies and actions.

We specifically are going to focus on the speeches and remarks of Ursula von der Leyen, the current President of the European Commission. The data collected will offer insights into the European Commission's stance and support for Ukraine.

In [ ]:
link = 'https://ec.europa.eu/commission/presscorner/home/en'
geko_path = '/Users/viktoriia/Desktop/BSE/Term 2/NLP/NLP-Project/geckodriver'
profile_path = '/Users/viktoriia/Library/Application Support/Firefox/Profiles/k7kr4dw0.Viktoriia'

## Scraping Pipeline

In [ ]:
# Initialize an empty dataframe to store the results
ec_articles_df = pd.DataFrame(columns=['Title', 'Date', 'Category', 'Location', 'Text'])

# Initialize type flag
first_type = True

# CSS for all types of documents we are interested in
document_types_css = {'STATEMENT' : '#filter-documentType > option:nth-child(13)', 
                  'SPEECH': '#filter-documentType > option:nth-child(12)',
                  'PRESS RELEASE': '#filter-documentType > option:nth-child(9)'}


for document_type in ['STATEMENT', 'SPEECH', 'PRESS RELEASE']:
    if first_type:
        # Initialize the scraper and open the main page
        scraper = s.TheEuropeanCommissionScraper()
        browser = scraper.start_up(link, geko_path, profile_path)
    else:
        # Open the main page
        browser = scraper.start_up(link, geko_path, profile_path, browser)

    browser = scraper.fill_in_filters(browser, document_type = document_types_css[document_type])    
    
    # Iterate the pages 
    while scraper.element_exists(browser, path='//a[@title="Go to next page"]', e_type='xpath'):
        # fix this, because we are missing the data from the last page
        browser, temp_list = scraper.get_articles(browser)
        df_temp = pd.DataFrame(temp_list)
        ec_articles_df = pd.concat([ec_articles_df, df_temp], ignore_index=True)
        browser.find_element('xpath','//a[@title="Go to next page"]').click()
        time.sleep(2)
    
    first_type = False

In [ ]:
ec_articles_df.shape

In [ ]:
ec_articles_df.head()

In [ ]:
# Save to csv
ec_articles_df.to_csv('theeuropeancommission.csv', index=False)